# main.ipynb
A notebook designing the substrate for the matching system

In [21]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [22]:
import pyphi
from substrate_modeler import unit, substrate
from matching.utils.convert import ces2df

from tqdm.auto import tqdm
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [23]:
pyphi.compute.parallel.init(address='144.92.2.41:99990')

In [25]:
pyphi.config.PROGRESS_BARS = False

## Build substrate


- LGN->V1: paralell inputs, mismatch detector
- within V1: Grid with self loop and lateral excitation that gets weaker with distance. units always help eachother maintain their states
- V1->V2: contiguity detector
- within V2: self loop (only a single detector)
- V2->V1: state-dependent modulation (if ON: shift threshold to the left; if OFF: shift threshold to the right)

In [5]:
input_layer = ["L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "L9"] 
layer_1 = ["A", "B", "C", "D", "E", "F", "G", "H", "I"] 
layer_2 = ["BCD", "CDE", "DEF", "EFG"] 
Layer_3 = ["V3"]

labels = input_layer + layer_1 + layer_2 + Layer_3
N = len(labels)
M = len(input_layer)

### Define LGN units
Each unit in the LGN has a self loop, and no other inputs. THey are modeled as simple sigmoids with a relatively low determinism. They will be outside the main entity anyways.

In [6]:
# parameters for LGN sigmoid
LGN_determinism = 3
LGN_threshold = 0.5
LGN_floor = 0.0
LGN_ceiling = 1.0

LGN_weights = [1.0]

# create the basic unit, which will be used to define each of the LGN units in our substrate. Thwy will just vary with regards to from whom they receive inputs, their label, and their index (see below)
LGN_unit = dict(
        input_weights=LGN_weights,
        determinism=LGN_determinism,
        threshold=LGN_threshold,
        floor=LGN_floor,
        ceiling=LGN_ceiling,
        ising = False
    )

# build the LGN units
LGN = [
    unit.Unit(
        index=i,
        label=labels[i],
        inputs=(i,),
        mechanism="sigmoid",
        params=LGN_unit,
        state=(0,),
        input_state =(0,)
    )
    for i in range(M)
]

# We now have a list of five identical (except their labels, inputs and indices) units that will form our input layer

### Define V1 units
V1 units are composed of two mechanisms, and they are modulated by V2.

First, they function as mismatch detectors with regards to the input from LGN. This means that they will be very strongly affected by inputs from the LGN if the LGN input is in a different state than the current state of the V1 unit. However, the LGN input has no effect on the state of the V1 unit if they are in the same state.

Second, the V1 units implement a function inspired by (or hypothesized to be found in) neocortex. Basically, units receive inputs from their (near and far) neighbors (strength dependent on distance, for example), but regardless off the states of the units, the inputs always contribut to keeping the V1 unit in its current state. This is implemented using a simple sigmoid function, where the inputs are coded using ising states and the sign of the connection weights depend on the states of the V1unit and the inputs. If the V1 unit and its input is in the same state, the input will be positive, but if they are in oposite states they will be engative (and weaker?).

Finally, the sigmoids are modulated by top down V2 input. The modulation is such that the threshold of the sigmoid is shifted depending on the state of the unit. If the unit is ON, then the threshold is shifted to the left (essentially increasing probability of firing). And oopposite for OFF

In [7]:
# parameters for sigmoidal subunit

# V1_determinism = 4
V1_determinism = 1

V1_threshold = 0.0
V1_floor = 0.01
V1_ceiling = 0.99

# define mapping to scale weight based on (unit_state, input_state)
V1_weight_scale_mapping = {
    (0, 0): 1.0,
    (1, 0): 0.5,
    (0, 1): 0.75,
    (1, 1): 1.5
}

self_loop = 0.5
lateral = 0.25
V1_weights = [
    lateral,  # near neighbor
    self_loop,  # self
    lateral,  # near neighbor
    0.01,  # V3
]

V1_sigmoid = dict(
        input_weights=V1_weights,
        determinism=V1_determinism,
        threshold=V1_threshold,
        weight_scale_mapping=V1_weight_scale_mapping,
        floor=V1_floor,
        ceiling=V1_ceiling,
)


# parameters for mismatch detector subunit# parameters for sigmoidal subunit
V1_mm_floor = 0.01
V1_mm_ceiling = 0.99
V1_input = dict(
    floor=V1_mm_floor,
    ceiling=V1_mm_ceiling,
    bias=0.0
)

In [8]:

# feedback amplification
feedback = (N-1,)
V1_sigmoid_inputs = (17,9,10,11,12,13,14,15,16,17,9)

V1 = [
    unit.CompositeUnit(
        index=M + i,
        label=labels[M + i],
        state=(0,),
        units=[
            unit.Unit(
                index=M + i,
                label = labels[M + i],
                state=(0,),
                inputs=V1_sigmoid_inputs[i : i + 3] + feedback,
                input_state=(0,)*4,
                mechanism="resonnator",#"sigmoid",#
                params = V1_sigmoid, 
            ),
            unit.Unit(
                index=M + i,
                label = labels[M + i],
                state=(0,),
                inputs = (i,),
                input_state=(0,),
                mechanism = "mismatch_corrector",
                params = V1_input,
            )
            ],  # FB_sigmoid
        mechanism_combination="selective",
    )
    for i in range(len(layer_1))
] 

# V2
The V2 units are segment detectors. They also have a sigmoidal self-loop.

Each detector is selective to a single segment state in V1 where exactly one contiuous segment is present. They implement selective "conjunction" mechanisms---only active when their preferred state is in the inputs. 


In [9]:
# Define sets of V1 states the V2 gate is selective to
V1_segments = [(0, 1, 1, 1, 0)]

In [10]:
# Gabor-like detectors
detector_floor = 0.01
detector_ceiling = 0.99
detector_selectivity = 5.0

V2_detector = dict(
    floor=detector_floor,
    ceiling=detector_ceiling,
    selectivity=detector_selectivity,
    pattern_selection=V1_segments,
)

# self loop for V2, based on a sigmoid
self_floor = 0.01
self_ceiling = 0.2
self_determinism = 1
self_threshold = 0.5

self_weights = [1.0, 0.5]

self_sigmoid = dict(
    input_weights=self_weights,
    determinism=self_determinism,
    threshold=self_threshold,
    weight_scale_mapping=V1_weight_scale_mapping,
    floor=self_floor,
    ceiling=self_ceiling,
)


In [11]:
first_i = len(input_layer) + len(layer_1)
layer_1_indices = tuple(range(M,M*2))

# combining the mechanisms into a composite unit
V2 = [
    unit.CompositeUnit(
        index=first_i + i,
        label=labels[first_i + i],
        state=(0,),
        units=[
            unit.Unit(
                index=first_i + i,
                label=labels[first_i + i],
                state=(0,),
                inputs=layer_1_indices[i+1:6+i],
                input_state=(0,)*5,
                mechanism="sor",
                params=V2_detector,
            ),
            unit.Unit(
                index=first_i + i,
                label=labels[first_i + i],
                state=(0,),
                inputs=(first_i + i,) + feedback,
                input_state=(0,)*2,
                mechanism="resonnator",
                params=self_sigmoid
            )
        ],
        mechanism_combination="serial",
    )
    for i in range(len(layer_2))
]

# V3
The V3 unit is a detector over the V2 units. It also has a sigmoidal self-loop.

The detector is selective to any state of the V2 units where exactly one of them is ON (indicating a segment is present in V1). It implements "disjunction" mechanism---active whenever one of its preferred states are on its inputs. 


In [12]:
# Define sets of V1 states the V2 gate is selective to
V2_states = [
    (1,0,0,0,),
    (0,1,0,0,),
    (0,0,1,0,),
    (0,0,0,1,),
]

In [13]:
# Gabor-like detectors
detector_floor = 0.01
detector_ceiling = 0.99
detector_selectivity = 5.0

V3_detector = dict(
    floor=detector_floor,
    ceiling=detector_ceiling,
    selectivity=detector_selectivity,
    pattern_selection=V2_states,
)

# self loop for V2, based on a sigmoid
self_floor = 0.01
self_ceiling = 0.2
self_determinism = 1
self_threshold = 0.5

self_weights = [1.0]

self_sigmoid = dict(
    input_weights=self_weights,
    determinism=self_determinism,
    threshold=self_threshold,
    weight_scale_mapping=V1_weight_scale_mapping,
    floor=self_floor,
    ceiling=self_ceiling,
)

In [14]:
layer_2_indices = tuple(range(first_i,first_i+4))

# combining the mechanisms into a composite unit
V3 = [
    unit.CompositeUnit(
        index=N-1,
        label=labels[N-1],
        state=(0,),
        units=[
            unit.Unit(
                index=N-1,
                label=labels[N-1],
                state=(0,),
                inputs=layer_2_indices,
                input_state=(0,)*4,
                mechanism="sor",
                params=V3_detector
            ),
            unit.Unit(
                index=N-1,
                label=labels[N-1],
                state=(0,),
                inputs=(N-1,),
                input_state=(0,),
                mechanism="resonnator",
                params=self_sigmoid
            )
        ],
        mechanism_combination="serial",
    )
]

In [15]:
units = LGN+V1+V2+V3

## create subsystems and compute some concepts

In [16]:
def create_subsystems(state, units):
    full_substrate = substrate.Substrate(units, state, implicit=True)
    V1 = full_substrate.isolate_subset(tuple(range(9,18)))
    V1_w_inputs = full_substrate.isolate_subset(tuple(range(18)))
    one_detector = full_substrate.isolate_subset((9,10,11,12,13,18))
    all_detectors = full_substrate.isolate_subset(tuple(range(9,23)))
    pyramid = full_substrate.isolate_subset(tuple(range(9,23)))
    return (
        full_substrate.subsystem(nodes=tuple(range(8,21))),
        V1.subsystem(),
        V1_w_inputs.subsystem(nodes=tuple(range(8,16))),
        one_detector.subsystem(),
        all_detectors.subsystem(),
        pyramid.subsystem()
    )

In [17]:
def concepts(
    subsystem,
    mechanisms,
    purviews=False,
    cause_purviews=False,
    effect_purviews=False
):
    if not purviews:
        purviews = [purviews]*len(mechanisms)
    if not cause_purviews:
        cause_purviews = [cause_purviews]*len(mechanisms)
    if not effect_purviews:
        effect_purviews = [effect_purviews]*len(mechanisms)
        
    return [
        subsystem.concept(
            mechanism,
            purview,
            cause_purview,
            effect_purview
        )
        for (
            mechanism,
            purview,
            cause_purview,
            effect_purview
        ) in tqdm(
            zip(mechanisms,purviews,cause_purviews,effect_purviews),
            total=len(mechanisms)
        )
    ]


### All off

In [18]:

mechanisms = [(5,),(4,5),(4,5,6),(3,4,5,6),(3,4,5,6,7),]#(2,3,4,5,6,7),(1,2,3,4,5,6,7)]

In [26]:
state = (0,)*23
(
    full_substrate,
    V1,
    V1_w_inputs,
    one_detector,
    all_detectors,
    pyramid
) = create_subsystems(state, units)

In [ ]:
cons = concepts(V1,mechanisms)
ces2df(pyphi.models.CauseEffectStructure(cons,subsystem=V1))

  0%|          | 0/5 [00:00<?, ?it/s]

### All on

In [ ]:
state = (1,)*23
(
    full_substrate,
    V1,
    V1_w_inputs,
    one_detector,
    all_detectors,
    pyramid
) = create_subsystems(state, units)

In [ ]:
cons_all_on = concepts(V1,mechanisms)
ces2df(pyphi.models.CauseEffectStructure(cons_all_on,subsystem=V1))

### segment ON

In [ ]:
state = (0,0,0,1,1,1,0,0,0)*2+(0,0,0,1,1)
(
    full_substrate,
    V1,
    V1_w_inputs,
    one_detector,
    all_detectors,
    pyramid
) = create_subsystems(state, units)

In [ ]:
cons_segment = concepts(V1,mechanisms)
ces2df(pyphi.models.CauseEffectStructure(cons_segment,subsystem=V1))

### heterogenous

In [ ]:
state = (0,1,0,1,0,1,0,1,0)*2+(0,0,0,0,0)
(
    full_substrate,
    V1,
    V1_w_inputs,
    one_detector,
    all_detectors,
    pyramid
) = create_subsystems(state, units)

In [ ]:
cons_hetero = concepts(V1,[(4,),(3,4),(2,3,4),(2,3,4,5),(2,3,4,5,6),(2,3,4,5,6,7)])
ces2df(pyphi.models.CauseEffectStructure(cons_hetero,subsystem=V1))

In [59]:
def create_substrates(state, units):
    full_substrate = substrate.Substrate(units, state, state, implicit=True)
    
    V1 = full_substrate.isolate_subset(tuple(range(9,18)))
    V1_w_inputs = full_substrate.isolate_subset(tuple(range(18)))
    one_detector = full_substrate.isolate_subset((9,10,11,12,13,18))
    all_detectors = full_substrate.isolate_subset(tuple(range(9,23)))
    pyramid = full_substrate.isolate_subset(tuple(range(9,23)))
    
    return (
        full_substrate,
        V1,
        V1_w_inputs,
        one_detector,
        all_detectors,
        pyramid
    )

In [64]:
state = (0,1,0,1,0,1,0,1,0)*2+(0,0,0,0,0)
state = (0,0,0,1,1,1,0,0,0)*2+(0,0,0,1,1)
state = (0,)*23
#state = (1,)*23
(
    full_substrate,
    V1,
    V1_w_inputs,
    one_detector,
    all_detectors,
    pyramid
) = create_substrates(state, units)

In [65]:
V1.units[3].tpm

ExplicitTPM(
[[[[0.27356704]
   [0.37999268]]

  [[0.5       ]
   [0.62000732]]]


 [[[0.37999268]
   [0.5       ]]

  [[0.62000732]
   [0.72643296]]]]
)

In [93]:

mechanisms = [(5,),(4,5),(4,5,6),(3,4,5,6),(3,4,5,6,7),]#(2,3,4,5,6,7),(1,2,3,4,5,6,7)]

In [77]:
V1.subsystem().concept((3,4,5,6))

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Distinction: mechanism = [D,E,F,G], state = [0, 0, 0, 0]
                  φ = 0.0579163007843                   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
                 MIC                                  MIE                 
┌───────────────────────────────────┐┌───────────────────────────────────┐
│  φ = 0.1257310066881              ││  φ = 0.0579163007843              │
│  Normalized φ = 0.0314327516720   ││  Normalized φ = 0.0289581503922   │
│  Purview: [E,F]                   ││  Purview: [E,F]                   │
│  Specified state:                 ││  Specified state:                 │
│  ┌───────────────────────────┐    ││  ┌────────────────────────────┐   │
│  │      Specified CAUSE      │    ││  │      Specified EFFECT      │   │
│  │ ╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍ │    ││  │ ╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍ │   │
│  │ CAUSE:  (0, 0)            │    ││  │ EFFECT:  (0, 0)    

In [78]:
V1.units[3].tpm

ExplicitTPM(
[[[[0.27356704]
   [0.37999268]]

  [[0.5       ]
   [0.62000732]]]


 [[[0.37999268]
   [0.5       ]]

  [[0.62000732]
   [0.72643296]]]]
)

In [34]:
ces2df(pyphi.models.CauseEffectStructure(cons,subsystem=V1))

,mechanism,mechanism_state,phi,cause_purview,cause_state,cause_phi,effect_purview,effect_state,effect_phi
0,D|,0,0.186485,D|,0,0.186485,D|,0,0.186485
1,C|D,00,0.109100,C|D,00,0.109100,C|D,00,0.114861
2,B|C|D,000,0.079727,C|,0,0.221913,C|,0,0.079727
3,B|C|D|E,0000,0.057916,C|D,00,0.125731,C|D,00,0.057916
4,B|C|D|E|F,00000,0.080312,C|D|E,000,0.080312,C|D|E,000,0.084145
5,B|C|D|E|F|G,000000,0.051032,C|D|E|F,0000,0.051032,C|D|E|F,0000,0.061125
